In [2]:
import cv2
import numpy as np
def length(start, end):
    return np.sqrt((start[0] - end[0]) ** 2 + (start[1] - end[1]) ** 2)
def get_angle(start, rotation_center, end):
    sr = length(start, rotation_center)
    re = length(rotation_center, end)
    se = length(start, end)
    cos_angle = (se ** 2 - sr **2 - re**2)/(-2 * sr * re)
    return np.arccos(cos_angle) / np.pi * 180


In [40]:
marker_img = cv2.imread("frame0.jpg", cv2.IMREAD_COLOR)
trochanter, trochanter_idx = marker_img[520:540, 85:115, :], 0
knee, knee_idx = marker_img[690:740, 335:395, :], 1
ankle, ankle_idx = marker_img[920:965, 120:150, :], 2
cv2.imshow('frame',knee)
cv2.waitKey(0)

markers = [trochanter, knee, ankle]

In [50]:
EXPORT_UNEDITED_FRAMES = [0]
EXPORT_EDITED_FRAMES = [0, 331, 365]
CROP_START_FRAMECOUNT = 150 # Set this to when you want to start the video
CROP_END_FRAMECOUNT = 800 # Set this to when you want to end the video
ROTATION = True # True if rotated to the right by 90deg
ROM_PIXELS = 50 # number of maximal pixels of movement between frames

COL_BLUE = (255, 0, 0)
COL_GREEN = (0, 255, 0)

ANGLE_COLOR = COL_GREEN

font                   = cv2.FONT_HERSHEY_SIMPLEX
fps_text = (30,50)
knee_angle_text = (30, 80)
fontScale              = 1
fontColor              = COL_BLUE
thickness              = 1
lineType               = 2

vid_name = 'Sattel-3mm.MOV'
vid = cv2.VideoCapture(vid_name)
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
width  = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = vid.get(cv2.CAP_PROP_FPS)
framecount = vid.get(cv2.CAP_PROP_FRAME_COUNT)
print (f"height:{height} width:{width}, fps: {fps}, framecount: {framecount}")

out = cv2.VideoWriter('output.avi', cv2.VideoWriter_fourcc(*"MJPG"), fps, (height, width))
count = 0
marker_coords = []
middle_points = []
# Read until video is completed
while(vid.isOpened()):
    # Capture frame-by-frame
    ret, frame = vid.read()
    if not ret:
        break
    if CROP_START_FRAMECOUNT <= count <= CROP_END_FRAMECOUNT:
        frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        cv2.putText(frame,f"Frame: {count}", fps_text, font, fontScale, fontColor, thickness, lineType)
        if count in EXPORT_UNEDITED_FRAMES:
            cv2.imwrite(f"frame{count}.jpg", frame)
        if count == CROP_START_FRAMECOUNT:
            for marker in markers:
                matched = cv2.matchTemplate(frame, marker, cv2.TM_SQDIFF_NORMED)
                marker_h, marker_w = marker.shape[:-1]
                _, _, top_left, _ = cv2.minMaxLoc(matched)
                middle_point = (top_left[0] + marker_w//2, top_left[1] + marker_h//2)
                bottom_right =  (top_left[0] + marker_w, top_left[1] + marker_h)
                cv2.rectangle(frame ,top_left, bottom_right, COL_BLUE, 2)
                cv2.circle(frame, middle_point, 5, COL_BLUE, -1)
                marker_coords.append([top_left])
                middle_points.append([middle_point])
        else:
            for (idx, marker) in enumerate(markers):
                last_coords = marker_coords[idx][-1]
                window_left = min(ROM_PIXELS, last_coords[0])
                window_right = min(ROM_PIXELS, frame.shape[1] - last_coords[0])
                window_top = min(ROM_PIXELS, last_coords[1])
                window_bottom = min(ROM_PIXELS, frame.shape[0] - last_coords[1])
                marker_h, marker_w = marker.shape[:-1]
                matched = cv2.matchTemplate(frame[last_coords[1]-window_top: last_coords[1]+marker_h + window_bottom, 
                                                  last_coords[0]-window_left: last_coords[0]+marker_w + window_right, :]
                                            , marker, cv2.TM_CCORR_NORMED )
                def transform_coordinates(hit_coordinates):
                    return (hit_coordinates[0] + last_coords[0]-window_left, hit_coordinates[1] + last_coords[1]-window_top)
                _, _, _, max_loc = cv2.minMaxLoc(matched)
                top_left = transform_coordinates(max_loc)
                middle_point = (top_left[0] + marker_w//2, top_left[1] + marker_h//2)
                bottom_right =  (top_left[0] + marker_w, top_left[1] + marker_h)
                cv2.rectangle(frame ,top_left, bottom_right, COL_BLUE, 2)
                cv2.circle(frame, middle_point, 5, COL_BLUE, -1)
                marker_coords[idx].append(top_left)
                middle_points[idx].append(middle_point)
                if idx == 1:
                    print(count, abs(last_coords[0] - top_left[0]) + abs(last_coords[1] - top_left[1]))
                    #cv2.imshow("Frame", matched)
                    #cv2.waitKey(1)
        # Write knee angle into the video:
        trochanter_pt = middle_points[trochanter_idx][-1]
        knee_pt = middle_points[knee_idx][-1]
        ankle_pt = middle_points[ankle_idx][-1]
        knee_angle = get_angle(trochanter_pt, knee_pt, ankle_pt)
        cv2.line(frame, trochanter_pt, knee_pt, ANGLE_COLOR, 2)
        cv2.line(frame, knee_pt, ankle_pt, ANGLE_COLOR, 2)
        cv2.putText(frame,f"Knee angle: {round(knee_angle, 2)}", knee_angle_text, font, fontScale, fontColor, thickness, lineType)
        if count in EXPORT_EDITED_FRAMES:
            cv2.imwrite(f"frame{count}marked.jpg", frame)
        out.write(frame)
        if count % 50 == 0:
            print(count)
    count += 1
 
# When everything done, release the video capture object
vid.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()

height:720 width:1280, fps: 50.0, framecount: 1043.0
150
151 33
152 30
153 29
154 21
155 18
156 9
157 3
158 3
159 9
160 12
161 19
162 23
163 31
164 35
165 40
166 42
167 43
168 38
169 35
170 30
171 21
172 13
173 33
174 31
175 10
176 14
177 19
178 23
179 27
180 31
181 32
182 34
183 34
184 35
185 35
186 32
187 29
188 24
189 17
190 9
191 2
192 8
193 14
194 20
195 27
196 33
197 39
198 42
199 45
200 44
200
201 40
202 35
203 28
204 18
205 11
206 61
207 4
208 73
209 17
210 21
211 25
212 30
213 33
214 35
215 36
216 36
217 35
218 33
219 29
220 23
221 18
222 9
223 1
224 10
225 14
226 21
227 25
228 33
229 38
230 43
231 44
232 44
233 37
234 32
235 23
236 17
237 8
238 54
239 13
240 62
241 16
242 20
243 26
244 29
245 33
246 33
247 36
248 37
249 34
250 34
250
251 28
252 22
253 17
254 6
255 1
256 11
257 16
258 21
259 29
260 34
261 40
262 47
263 45
264 45
265 38
266 30
267 25
268 16
269 32
270 1
271 10
272 15
273 12
274 9
275 12
276 8
277 22
278 25
279 59
280 58
281 17
282 14
283 24
284 14
285 16
286 6
